# Automation Excel to CSV and GeoJSON

## 1. Import Library

In [32]:
import pandas as pd
from openpyxl import load_workbook
import geopandas as gpd
from shapely.geometry import Point
import os

## 2. Application to Export Excel into CSV

### 2.1. Function Codes

In [33]:
def flatten_excel(file_path, output_folder):
    # Load Excel workbook
    wb = load_workbook(file_path, data_only=True)
    
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Unmerge cells and fill values
        for merge in list(ws.merged_cells):
            ws.unmerge_cells(str(merge))
            top_left = ws.cell(merge.min_row, merge.min_col).value
            for row in range(merge.min_row, merge.max_row + 1):
                for col in range(merge.min_col, merge.max_col + 1):
                    ws.cell(row, col, top_left)

        # Convert to DataFrame
        data = list(ws.values)
        df = pd.DataFrame(data)

        # Identify the header row (where "NO" appears)
        header_index = df[df.apply(lambda x: x.astype(str).str.contains("NO", case=False, na=False)).any(axis=1)].index[0]

        # Use row 3 (index 2) as the header
        df.columns = df.iloc[2].astype(str).str.strip()

        # Remove empty columns
        df = df.dropna(axis=1, how="all")

        # Drop the "REKAP" section if present
        df = df.loc[:, ~df.columns.str.contains("REKAP", case=False, na=False)]
        df = df.drop(index=[0, 1, 4]).reset_index(drop=True)

        # Merge the first two rows: Keep if same, else merge
        merged_header = [a if a == b else f"{a} {b}" for a, b in zip(df.iloc[0], df.iloc[1])]

        # Update DataFrame headers and remove first two rows
        df.columns = merged_header
        df = df.drop(index=[0, 1]).reset_index(drop=True)

        # Construct full path for CSV file
        csv_filename = os.path.join(output_folder, f"{sheet_name}.csv")
        
        # Save DataFrame as CSV
        df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

        print(f"✅ {sheet_name} exported to {csv_filename}")

    print(f"✅ All sheets processed successfully! Files saved in: {output_folder}")

### 2.2. Run Function

In [34]:
excel_file = r"C:\Users\kanzi\Documents\Part Time Job\Automation Codes\02. CILEUNGSI - CIBINONG (CITEUREUP).xlsx"  # Fill with the path file of excel
export_folder = r"C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2"  # Fill with the path folder of export result
flatten_excel(excel_file, export_folder) # Run the function!

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


✅ RAMBU exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\RAMBU.csv
✅ PJU exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\PJU.csv
✅ RPPJ exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\RPPJ.csv
✅ PAGAR PENGAMAN exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\PAGAR PENGAMAN.csv
✅ MARKA exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\MARKA.csv
✅ WARNING LIGHT exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\WARNING LIGHT.csv
✅ APILL exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\APILL.csv
✅ ZOSS exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\ZOSS.csv
✅ FAS PENYEBERANGAN exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\FAS PENYEBERANGAN.csv
✅ RAMBU PORTABLE exported to C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\RAMBU PORTABLE.csv
✅ 

## 3. Application to Export Excel into GeoJson

### 3.1. Function Codes

In [ ]:
def make_column_names_unique(columns): #Function to ensure column names are unique by appending '_1', '_2'
    seen = {}
    new_columns = []
    
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")  # Append a counter to duplicate columns
        else:
            seen[col] = 0
            new_columns.append(col)
    
    return new_columns

def clean_column_names(columns): #Standardize column names by capitalizing each word properly.
    cleaned_columns = []
    seen = {}

    for col in columns:
        col = str(col).strip()  # Remove extra spaces
        col = " ".join(word.capitalize() for word in col.split())  # Capitalize each word
        
        # Ensure uniqueness by appending a counter if needed
        if col in seen:
            seen[col] += 1
            col = f"{col} {seen[col]}"
        else:
            seen[col] = 0
        cleaned_columns.append(col)

    return cleaned_columns

def flatten_excel_to_geojson(file_path, output_folder): 
#Convert all sheets from an Excel file to GeoJSON, ensuring unique column names, drop unuse column, fix bad format of coordinate
    
    # Load workbook
    wb = load_workbook(file_path, data_only=True)

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Unmerge cells and fill values
        for merge in list(ws.merged_cells):
            ws.unmerge_cells(str(merge))
            top_left = ws.cell(merge.min_row, merge.min_col).value
            for row in range(merge.min_row, merge.max_row + 1):
                for col in range(merge.min_col, merge.max_col + 1):
                    ws.cell(row, col, top_left)

        # Convert to DataFrame
        data = list(ws.values)
        df = pd.DataFrame(data)

        # Identify the header row
        header_index = df[df.apply(lambda x: x.astype(str).str.contains("NO", case=False, na=False)).any(axis=1)].index[0]

        # Use row 3 (index 2) as the header
        df.columns = df.iloc[2].astype(str).str.strip()

        # Remove empty columns
        df = df.dropna(axis=1, how="all")

        # Drop "REKAP" section if present
        df = df.loc[:, ~df.columns.str.contains("REKAP", case=False, na=False)]
        df = df.drop(index=[0, 1, 4]).reset_index(drop=True)

        # Merge first two rows if needed
        merged_header = [a if a == b else f"{a} {b}" for a, b in zip(df.iloc[0], df.iloc[1])]

        # Ensure column names are unique
        df.columns = make_column_names_unique(merged_header)

        # Remove the first two rows used for headers
        df = df.drop(index=[0, 1]).reset_index(drop=True)

        # Normalize column names for consistent detection
        df.columns = df.columns.str.lower().str.strip()

        # Find Latitude & Longitude columns dynamically
        lat_col = next((col for col in df.columns if "latitude" in col), None)
        lon_col = next((col for col in df.columns if "longitude" in col), None)

        if not lat_col or not lon_col:
            print(f"⚠️ Skipping '{sheet_name}' (No Latitude/Longitude columns)")
            continue  # Skip this sheet if Lat/Lon are missing

        # Convert Lat/Lon to numeric first (forcing errors to NaN)
        df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
        df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')

        # Fix latitude and longitude values
        def fix_coordinates(row):
            """Corrects lat/lon values if they are missing decimal points."""
            lat, lon = row[lat_col], row[lon_col]
            
            # Ensure values are numeric before applying conditions
            if pd.notna(lat) and abs(lat) > 90:  # Latitude should be between -90 and 90
                lat /= 1_000_000
            if pd.notna(lon) and abs(lon) > 180:  # Longitude should be between -180 and 180
                lon /= 1_000_000
            
            return pd.Series([lat, lon])

        # Apply the fix function
        df[[lat_col, lon_col]] = df.apply(fix_coordinates, axis=1)

        # Remove rows where Lat/Lon are still missing
        df = df.dropna(subset=[lat_col, lon_col]).reset_index(drop=True)

        # Create GeoDataFrame
        properties_cols = [col for col in df.columns if col not in [lat_col, lon_col]]
        geometry = [Point(xy) if pd.notna(xy[0]) and pd.notna(xy[1]) else None for xy in zip(df[lon_col], df[lat_col])]

        gdf = gpd.GeoDataFrame(df[properties_cols], geometry=geometry, crs="EPSG:4326")

        # Apply column renaming after creating the GeoDataFrame
        gdf.columns = clean_column_names(gdf.columns)

        # Drop invalid geometries
        gdf = gdf.dropna(subset=['Geometry'])

        # Ensure all column names are strings
        gdf.columns = gdf.columns.astype(str)

        # Remove unwanted "None_" and "None" columns
        gdf = gdf.loc[:, ~gdf.columns.str.match(r"^None$|None_", na=False)]

        # Remove " None" from remaining column names
        gdf.columns = gdf.columns.str.replace(r"\sNone\b", "", regex=True).str.strip()

        # Create output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Define output file path
        output_path = os.path.join(output_folder, f"{sheet_name}.geojson")

        # Export to GeoJSON
        gdf.to_file(output_path, driver="GeoJSON")

        print(f"✅ Saved: {output_path}")

    print("🎉 All sheets processed successfully!")

### 3.2. Run Function

In [119]:
excel_file = r"C:\Users\kanzi\Documents\Part Time Job\Automation Codes\02. CILEUNGSI - CIBINONG (CITEUREUP).xlsx"  # Fill with the path file of excel
export_folder = r"C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2"  # Fill with the path folder of export result
flatten_excel_to_geojson(excel_file, export_folder) # Run the function!

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\RAMBU.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\PJU.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\RPPJ.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\PAGAR PENGAMAN.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\MARKA.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\WARNING LIGHT.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\APILL.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\ZOSS.geojson
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Automation Codes\check2\FAS PENYEBERANGAN.geojson
⚠️ Skipping 'RAMBU PORTABLE' (No Latitude/Longitude columns)
⚠️ Skipping 'TRAFFIC CONE' (No Latitude/Longitude columns)
⚠️ Skipping 'WATER BARRIER' (No Latitude/Longitude columns)
✅ Saved: C:\Users\kan